In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import numpy as np
import glob
import cv2
import h5py
import pickle as pkl

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Activation
from keras.layers import ZeroPadding2D
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D

from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.utils import np_utils 
from keras import backend as K

from matplotlib import pyplot
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


## Training Stage

In [7]:
# Read images and labels to form dataset

# Labels are: 0-9, A(10), B(11), C(12), F(13), G(14), T(15), ruido(16)

# Insert dataset path and number of files to be read in each category
datapath = 'D:\\UFES\\RevisionWorkstation\\TrainingFiles\\TopHatTrainingFiles\\'
nfiles = [1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500]

filenames = []
for i in range(0,17):    
    files = [fname for fname in glob.glob(datapath + str(i) + '\\*.png')]
    files.sort
    filenames = np.hstack((filenames, files[0:int(nfiles[i])]))


filenames.sort
len(filenames)

24100

In [10]:
# Read images and labels to form dataset
filtering = True

images = []
rotulos = []

if filtering:

    input_shape = (180,120,1)
    for fname in filenames:

        # Get images
        img = cv2.imread(fname, 0) 
        if img is not None:
            img = cv2.threshold(img, 10, 255, cv2.THRESH_BINARY)[1]

            # Get classification (using the folder name)
            splits = fname.rsplit('\\', 2)

            # Uptade Dataset
            images.append(img)
            rotulos.append(int(splits[1]))

if not filtering:
     
    input_shape = (180,120,3)
    for fname in filenames:

        # Get images
        img = cv2.imread(fname, 1) 
        if img is not None:
          
            # Get classification (using the folder name)
            splits = fname.rsplit('\\', 2)

            # Uptade Dataset
            images.append(img)
            rotulos.append(int(splits[1]))
            
print('Total number of images: ' + str(len(images)))    

Total number of images: 24100


In [18]:
# Transform arrays
if filtering:
    images = np.array(images).reshape((len(images), 180, 120, 1))
    rotulos = np.array(rotulos).reshape((len(rotulos), 1))
if not filtering:
    images = np.array(images).reshape((len(images), 180, 120, 3))
    rotulos = np.array(rotulos).reshape((len(rotulos), 1))
    
# Normalize images
images = images.astype('float32')
rotulos = rotulos.astype('float32')

images /= 255

# Divide set to do single Holdout validation 
X_train = []
y_train = []
X_test = []
y_test = []
X_valid = []
y_valid = []
    
# X_train (50%), X_valid(25%), X_test (25%)
X_train, X_test, y_train, y_test = train_test_split(images, rotulos, test_size=0.25)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.33)    

# Converts a class vector (integers) to binary class matrix
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
y_valid = np_utils.to_categorical(y_valid)

In [21]:
# Define parameters
K.set_image_dim_ordering('th')
seed = 7
np.random.seed(seed)
num_classes = 17

In [22]:
# Create the model

arquitetura = 'Cifar'
#arquitetura = 'VGG'
#arquitetura = 'LeNet5'
#arquitetura = 'AlexNet'

if arquitetura == 'Cifar':

    # Parameters
    epochs = 20 
    lrate = 0.01
    decay = lrate/epochs
    sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
    
    # Model
    model = Sequential()
    
    model.add(Conv2D(16, (3,3), input_shape=input_shape, padding = 'same', activation = 'relu', data_format='channels_last'))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(16, (3,3), padding = 'same', activation = 'relu', data_format='channels_last'))
    model.add(MaxPooling2D(pool_size=(2,2), data_format='channels_last'))
    
    model.add(Conv2D(32,(3,3), padding = 'same', activation = 'relu', data_format='channels_last'))
    model.add(MaxPooling2D(pool_size=(2,2), data_format='channels_last'))
    
    model.add(Flatten())
    
    model.add(Dropout(0.2))
    model.add(Dense(512,activation='relu',kernel_constraint=maxnorm(3)))
    
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

if arquitetura == 'LeNet5':
    #https://github.com/TaavishThaman/LeNet-5-with-Keras/blob/master/lenet_5.py
    
    # Parameters
    epochs = 20
    steps_per_epoch = 10
    batch_size = 32
    
    # Model
    model = Sequential()
    
    model.add(Conv2D(12, (5, 5), input_shape=input_shape, padding = 'same', data_format='channels_last'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2,2), padding = 'same'))
   
    model.add(Conv2D(25, (5, 5), padding = 'same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2,2), padding = 'same'))
    
    model.add(Flatten())
    
    model.add(Dense(units = 180, activation = 'relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(units = 100, activation = 'relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
if arquitetura == 'AlexNet':
    #https://github.com/eweill/keras-deepcv/blob/master/models/classification/alexnet.py
    
    # Parameters
    l2_reg=0.
    
    # Model
    model = Sequential()

    model.add(Conv2D(96, (11, 11), input_shape=input_shape, padding='same', kernel_regularizer=l2(l2_reg)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding = 'same'))


    model.add(Conv2D(256, (5, 5), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding = 'same'))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding = 'same'))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(1024, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(1024, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding = 'same'))

    model.add(Flatten())
    model.add(Dense(3072))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(4096))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(num_classes))
    model.add(BatchNormalization())
    model.add(Activation('softmax'))
    
    model.compile(loss= 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
        
print('Arquitetura: ' + arquitetura)
print(model.summary())

Arquitetura: original
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 180, 120, 16)      160       
_________________________________________________________________
dropout_6 (Dropout)          (None, 180, 120, 16)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 180, 120, 16)      2320      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 90, 60, 16)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 90, 60, 32)        4640      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 45, 30, 32)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 43200)            

In [23]:
# Select place and name to save model
modelName = 'D:\\UFES\\RevisionWorkstation\\modelCifarClahe'

# Fit model to data set
model.fit(X_train, y_train, validation_data = (X_valid, y_valid), epochs=epochs, batch_size=batch_size)
    
# Evaluate model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

# Save model
model.save(modelName)

Train on 10979 samples, validate on 5409 samples
Epoch 1/20
10979/10979 [==============================] - 590s 54ms/step - loss: 0.1932 - acc: 0.9485 - val_loss: 0.1159 - val_acc: 0.9856
Epoch 2/20
10979/10979 [==============================] - 588s 54ms/step - loss: 0.0334 - acc: 0.9919 - val_loss: 0.0359 - val_acc: 0.9926
Epoch 3/20
10979/10979 [==============================] - 649s 59ms/step - loss: 0.0209 - acc: 0.9940 - val_loss: 0.0482 - val_acc: 0.9937
Epoch 4/20
10979/10979 [==============================] - 676s 62ms/step - loss: 0.0123 - acc: 0.9972 - val_loss: 0.0249 - val_acc: 0.9945
Epoch 5/20
10979/10979 [==============================] - 758s 69ms/step - loss: 0.0090 - acc: 0.9974 - val_loss: 0.0232 - val_acc: 0.9950
Epoch 6/20
10979/10979 [==============================] - 765s 70ms/step - loss: 0.0056 - acc: 0.9981 - val_loss: 0.0231 - val_acc: 0.9952
Epoch 7/20
10979/10979 [==============================] - 629s 57ms/step - loss: 0.0068 - acc: 0.9982 - val_loss: 0.0

## Comparing

In [ ]:
from keras.models import load_model

alexsNet = load_model('D:\\UFES\\RevisionWorkstation\\modelAlexNetNoFilter')
cifarNet = load_model('D:\\UFES\\RevisionWorkstation\\modelCifarNoFilter')
lenetNet = load_model('D:\\UFES\\RevisionWorkstation\\modelLeNet5NoFilter')

print("done loading")

In [9]:
scores1 = alexsNet.evaluate(X_valid,y_valid,verbose=0)
print("AccuracyAN: %.2f%%" % (scores1[1]*100))

scores2 = cifarNet.evaluate(X_valid,y_valid,verbose=0)
print("AccuracyCF: %.2f%%" % (scores2[1]*100))

scores3 = lenetNet.evaluate(X_valid,y_valid,verbose=0)
print("AccuracyLN: %.2f%%" % (scores3[1]*100))

AccuracyAN: 98.70%
AccuracyCF: 98.80%
AccuracyLN: 98.77%


## Aux Testing

In [ ]:
img = cv2.imread("D:\\Thais Caldeira\\Imagens Bobinas\\Training Files\\9\\file_502.jpg", 0) # gray
img = cv2.threshold(img,10,255,cv2.THRESH_BINARY)[1]

img = np.array(img).reshape((1, 180, 120, 1))

img = img.astype('float32')

img /= 255

scores = model.predict(img)
print(np.argmax(scores))
scores = np.array(scores).reshape(16,1)
print(scores)

In [ ]:
chlib = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'F', 'G']
